# Bibliotecas 


In [39]:
#pip install numpy pandas matplotlib seaborn scikit-learn imbalanced-learn torch optuna joblib tqdm

In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, precision_score, recall_score, 
    f1_score, accuracy_score, roc_curve, auc, make_scorer
)
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from datetime import datetime, timedelta
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import logging
import warnings
from torch.optim import lr_scheduler
from joblib import Parallel, delayed
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm



########################### para KAGGLE ################################################################################################################
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
########################################################################################################################################################

# Dicinário de Dados 


| **Variável**         | **Tipo**   | **Descrição**                                                                                                                                     |
|-----------------------|------------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| RowNumber            | int64      | Número do registro (linhas), sem efeito na construção de modelos.                                                                                |
| CustomerId           | int64      | ID do cliente, sem efeito sobre o estudo.                                                                                                       |
| Surname              | object     | Sobrenome do cliente, sem impacto na análise.                                                                                                   |
| CreditScore          | int64      | Pontuação de crédito, pode indicar tendência de permanência de clientes com pontuação alta.                                                     |
| Geography            | object     | Localização do cliente, pode influenciar a decisão de evasão.                                                                                   |
| Gender               | object     | Gênero do cliente, possível influência na evasão.                                                                                               |
| Age                  | int64      | Idade do cliente, clientes mais velhos tendem a permanecer.                                                                                     |
| Tenure               | int64      | Anos que o cliente está no banco, clientes novos têm maior chance de evasão.                                                                    |
| Balance              | float64    | Saldo na conta, pessoas com saldos altos são menos propensas a sair.                                                                            |
| NumOfProducts        | int64      | Número de produtos adquiridos pelo cliente.                                                                                                    |
| HasCrCard            | int64      | Indica se o cliente tem cartão de crédito, clientes com cartão são menos propensos à evasão.                                                    |
| IsActiveMember       | int64      | Clientes ativos têm menor chance de evasão.                                                                                                    |
| EstimatedSalary      | float64    | Salário estimado, clientes com salários mais altos tendem a permanecer.                                                                         |
| Exited               | int64      | Indica se o cliente saiu ou não do banco, variável de predição (“churn”).                                                                       |
| Complain             | int64      | Indica se o cliente fez reclamação.                                                                                                             |
| Satisfaction Score   | int64      | Pontuação de satisfação com a resolução de reclamação.                                                                                          |
| Card Type            | object     | Tipo de cartão que o cliente possui.                                                                                                            |
| Points Earned        | int64      | Pontos ganhos pelo cliente.                                                                                                                     |


#  Análise Exploratória (EDA) & Data Prep


In [ ]:
# base de dados
# base_original = pd.read_csv('/kaggle/input/Customer-Churn-Records.csv', sep=',') #KAGGLE
base_original = pd.read_csv('C:/Users/jgeov/iCloudDrive/Treinamento/Treinamento Data Science/Projetos/Customer-Churn-Records.csv',sep=',') #LOCAL

#configs para nao quebrar linhas no print do  df
pd.set_option('display.expand_frame_repr', False) 
pd.set_option('display.max_columns', None)

#primeiras linhas 
base_original.head()

In [ ]:
#Dimensões da base de dados
print("Numero de linhas:", base_original.shape[0]) 
print("Numero de colunas:", base_original.shape[1])

In [ ]:
#Verificando tipos dados originais 
base_original.dtypes

In [ ]:
#checando se há valores nulos 
base_original.isnull().sum()  
#valores nulos nao encontrados 

In [ ]:
# Resumo estatistico da base original
base_original.describe()


In [ ]:
#Limpando variavéis que não tem interferencia na analise, sao meramente
#identificadoras: RowNumber, CustomerId e Surname

df = base_original[['CreditScore',
                    'Gender',
                    'Geography',
                    'Age','Tenure',
                    'Balance',
                   'NumOfProducts',
                    'HasCrCard',
                    'IsActiveMember',
                   'EstimatedSalary',
                    'Complain',
                    'Satisfaction Score',
                   'Card Type',
                    'Point Earned',
                    'Exited'
                   ]]

# Resumo estatístico das variáveis quantitativas
quanti = df[['EstimatedSalary', 'Balance', 'CreditScore', 'Age', 'Tenure', 'Point Earned']]
resumo_estati_quant = quanti.describe().style.format(lambda x: f'{x:,.1f}'.replace(',', 'X').replace('.', ',').replace('X', '.')) # Formatação com 1 casa decimal e separadores invertidos

resumo_estati_quant

In [ ]:
#%% Resumo estatistico das varaiveis Quali ("tabelas" de frequencias)

quali = df[['HasCrCard', 'IsActiveMember', 'Geography','Gender'
            ,'Complain','Exited','Card Type']]

quali = quali.astype('object')

print("------------------------------------------")
print(quali['HasCrCard'].value_counts())
print("------------------------------------------")
print(quali['IsActiveMember'].value_counts())
print("------------------------------------------")
print(quali['Geography'].value_counts())
print("------------------------------------------")
print(quali['Gender'].value_counts())
print("------------------------------------------")
print(quali['Complain'].value_counts())
print("------------------------------------------")
print(quali['Card Type'].value_counts())
print("------------------------------------------")
print(quali['Exited'].value_counts())
print("------------------------------------------")

In [ ]:
# Verificando tipos das variaveis quali (para morrer de certeza) 
quali.dtypes

In [ ]:
# Visualizando Frequencia das variaveis categoricas 

def add_value_labels(ax):
    for p in ax.patches:
        height = p.get_height()
        color = p.get_facecolor()
        ax.text(p.get_x() + p.get_width() / 2., height / 2.,
                f'{int(height)}',
                ha='center', va='center', fontsize=20, color='white', fontweight='bold',
                bbox=dict(facecolor=color, edgecolor='none', alpha=0.7,
                          boxstyle='round,pad=0.4', linewidth=1))

plt.figure(figsize=(20, 25))

# Geography
plt.subplot(5, 2, 1)
ax1 = plt.gca()
ax1.set_title('Geography', fontsize=22, fontweight='bold')
sns.countplot(x='Geography', hue='Geography', palette='viridis', data=base_original, ax=ax1, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax1)

# Gender
plt.subplot(5, 2, 2)
ax2 = plt.gca()
ax2.set_title('Gender', fontsize=22, fontweight='bold')
sns.countplot(x='Gender', hue='Gender', palette='viridis', data=base_original, ax=ax2, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax2)

# Complain
plt.subplot(5, 2, 3)
ax10 = plt.gca()
ax10.set_title('Complain', fontsize=22, fontweight='bold')
sns.countplot(x='Complain', hue='Complain', palette='viridis', data=base_original, ax=ax10, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax10)

# HasCrCard
plt.subplot(5, 2, 4)
ax5 = plt.gca()
ax5.set_title('HasCrCard', fontsize=22, fontweight='bold')
sns.countplot(x='HasCrCard', hue='HasCrCard', palette='viridis', data=base_original, ax=ax5, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax5)

# IsActiveMember
plt.subplot(5, 2, 5)
ax6 = plt.gca()
ax6.set_title('IsActiveMember', fontsize=22, fontweight='bold')
sns.countplot(x='IsActiveMember', hue='IsActiveMember', palette='viridis', data=base_original, ax=ax6, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax6)

# Card Type
plt.subplot(5, 2, 6)
ax10 = plt.gca()
ax10.set_title('Card Type', fontsize=22, fontweight='bold')
sns.countplot(x='Card Type', hue='Card Type', palette='viridis', data=base_original, ax=ax10, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax10)

# Exited
plt.subplot(5, 2, 7)
ax7 = plt.gca()
ax7.set_title('Exited: churn variable', fontsize=22, fontweight='bold')
custom_palette = ['green', 'red']
sns.countplot(x='Exited', hue='Exited', palette=custom_palette, data=base_original, ax=ax7, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax7)

ax7.set_xticks([0, 1])
ax7.set_xticklabels(['Não', 'Sim'], fontsize=15, fontweight='bold')

# Ajustando espaçamento
plt.subplots_adjust(hspace=0.3, wspace=0.1)

plt.show()




In [ ]:
#%% Visualizando a Variável alvo em relação as demais variáveis 

plt.figure(figsize=(20, 25)) #tamanho do painel grafico

#funcao de adicao de legenda no canto superior direito e garante rotulos 
def add_legend(ax):
    
    handles, labels = ax.get_legend_handles_labels()
    if not handles:
        
        # Se não houver handles, forca a adicao
        handles = [plt.Rectangle((0,0),1,1, color=c) for c in ['green', 'red']]
        labels = ['Not Exited', 'Exited']
        
    # Adiciona a legenda fora da área das barras
    ax.legend(handles, labels, loc='upper left', fontsize=14, title='Exited', title_fontsize='13',  
              bbox_to_anchor=(1.0, 1)) 

# Geography
plt.subplot(5, 2, 1)
counts = base_original.groupby(['Geography', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by Geography', fontsize=22, fontweight='bold')
plt.xlabel('Geography', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Gender
plt.subplot(5, 2, 2)
counts = base_original.groupby(['Gender', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca()) 
plt.title('Exited by Gender', fontsize=22, fontweight='bold')
plt.xlabel('Gender', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# NumOfProducts
plt.subplot(5, 2, 3)
counts = base_original.groupby(['NumOfProducts', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by NumOfProducts', fontsize=22, fontweight='bold')
plt.xlabel('NumOfProducts', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# HasCrCard
plt.subplot(5, 2, 4)
counts = base_original.groupby(['HasCrCard', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by HasCrCard', fontsize=22, fontweight='bold')
plt.xlabel('HasCrCard', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# IsActiveMember
plt.subplot(5, 2, 5)
counts = base_original.groupby(['IsActiveMember', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca()) 
plt.title('Exited by IsActiveMember', fontsize=22, fontweight='bold')
plt.xlabel('IsActiveMember', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Complain
plt.subplot(5, 2, 6)
counts = base_original.groupby(['Complain', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by Complain', fontsize=22, fontweight='bold')
plt.xlabel('Complain', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Satisfaction Score
plt.subplot(5, 2, 7)
counts = base_original.groupby(['Satisfaction Score', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by Satisfaction Score', fontsize=22, fontweight='bold')
plt.xlabel('Satisfaction Score', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Card Type
plt.subplot(5, 2, 8)
counts = base_original.groupby(['Card Type', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by Card Type', fontsize=22, fontweight='bold')
plt.xlabel('Card Type', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Ajustar a distância entre os gráficos
plt.subplots_adjust(hspace=0.7, wspace=0.3)

# Variaveis Dummies e Correlações 

In [ ]:
# Dumizando

# Suprime todos os warnings de futuro (deixa mais clean) 
warnings.filterwarnings('ignore', category=FutureWarning)



# Lista de colunas a serem transformadas
cols_to_transform = ['HasCrCard', 'IsActiveMember', 'Geography', 'Gender', 'Card Type']

# Convertendo para string
df.loc[:, cols_to_transform] = df.loc[:, cols_to_transform].astype(str)

# one-hot encoding 
df_dummies = pd.get_dummies(df, columns=cols_to_transform, dtype=int, drop_first=False)

# variável alvo 'Exited' para o tipo numérico
df_dummies['Exited'] = df_dummies['Exited'].astype('int64')

print(df_dummies.dtypes)

In [ ]:
#Correlação das variaveis Numéricas 

# Matrix com Mapa de calor 
correlation_matrix = df_dummies.corr().round(2)
correlation_matrix

plt.figure(figsize=(30, 20))
heatmap = sns.heatmap(correlation_matrix, annot=True, fmt=".2f",
                      cmap=plt.cm.Blues,
                      annot_kws={'size': 15}, vmin=-1, vmax=1)
heatmap.set_xticklabels(heatmap.get_xticklabels(), fontsize=17)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=17)
cbar = heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=17)
plt.title('Correlação das Variáveis Quantitativas', fontsize=25)
plt.show()



# As variaveis a seguir apresentaram correlacao alta, para evitar multicolinearidade foram removidas 
df_dummies = df_dummies.drop(columns=['HasCrCard_0','IsActiveMember_0','Gender_Female','Complain'])

# Separação Treino e Teste

In [ ]:
#X ---> Variáveis explicativas 
#Y ---> Evento de estudo (variável TARGET, evento de estudo, ^y etc..)

X = df_dummies .drop('Exited', axis=1)
y =  df_dummies['Exited']


#separando em treino e teste 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Visualizando a proporção de eventos de churn (TARGET) nas bases de TREINO e TESTE 

# Contando os valores 
churn_counts_train = y_train.value_counts()
churn_counts_test = y_test.value_counts()


# Criando o plot 
fig, axs = plt.subplots(1, 2, figsize=(20, 10))
# Adicionando título 
fig.suptitle('Proporção da Variável Churn entre Treino e Teste', fontsize=35)
#definindo a paleta de cor 
cmap = plt.get_cmap('viridis', 2)



# Gráfico da base de treino
bars_train = axs[0].bar(churn_counts_train.index, churn_counts_train.values, color=cmap(range(2)))
axs[0].set_title('Base de Treino', fontsize=25)
axs[0].set_xlabel('Churn', fontsize=20)
axs[0].set_ylabel('Contagem', fontsize=20)
axs[0].set_xticks([0, 1])
axs[0].set_xticklabels(['0', '1'], fontsize=20)
# Ocultando os valores do eixo y
axs[0].set_yticklabels([])

# Adicionando rótulos de dados nas barras da base de treino com valor absoluto e percentual
total_train = churn_counts_train.sum()
for bar in bars_train:
    count = int(bar.get_height())
    percentage = round(count / total_train * 100)  # Arredonda a porcentagem
    label = f'{count} ({percentage}%)'  # Exibe o valor absoluto e o percentual
    axs[0].text(bar.get_x() + bar.get_width() / 2, bar.get_height() / 2, 
                label, ha='center', color='gray', fontsize=25, weight='bold')




# Gráfico da base de teste
bars_test = axs[1].bar(churn_counts_test.index, churn_counts_test.values, color=cmap(range(2)))
axs[1].set_title('Base de Teste', fontsize=25)
axs[1].set_xlabel('Churn', fontsize=20)
axs[1].set_ylabel('Contagem', fontsize=20)
axs[1].set_xticks([0, 1])
axs[1].set_xticklabels(['0', '1'], fontsize=20)
# Ocultando os valores do eixo y
axs[1].set_yticklabels([])

# Adicionando rótulos de dados nas barras da base de teste com valor absoluto e percentual
total_test = churn_counts_test.sum()
for bar in bars_test:
    count = int(bar.get_height())
    percentage = round(count / total_test * 100)  # Arredonda a porcentagem
    label = f'{count} ({percentage}%)'  # Exibe o valor absoluto e o percentual
    axs[1].text(bar.get_x() + bar.get_width() / 2, bar.get_height() / 2, 
                label, ha='center', color='gray', fontsize=25, weight='bold')



# Ajustar o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Deixa espaço para o título principal
plt.show()

In [ ]:
#Testando Multicolinearidade na base de treino

#as variaveis com alto correlacao foram removidas na primeira plotagem de matriz de corr, isso reduz o ruido no modelo

# todo esse processo de tirar as variaveis que tem alta correlacao com a variavel target deve ser analisado se realmente é necessario em cada cenario
# existem modelos que capturam bem isso e não são afetados pela multicolinearidade (pelo menos lidam bem) 

teste_multco_treino = pd.concat([X_train,y_train], axis = 1)

correlation_matrix_treino = teste_multco_treino.corr().round(2)
correlation_matrix_treino

# Matrix com  mapa de calor 
plt.figure(figsize=(30, 20))
heatmap = sns.heatmap(correlation_matrix_treino, annot=True, fmt=".2f",
                      cmap=plt.cm.viridis_r, # paleta de cores viridis (ou viridis_r para o inverso de cores) é uma paleta especial 
                                             # para facilitar a visualizacao por pessoas com dificuldades visuais, como os daltonicos. 
                      annot_kws={'size': 15}, vmin=-1, vmax=1)
heatmap.set_xticklabels(heatmap.get_xticklabels(), fontsize=17)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=17)
plt.title('Correlação das Variáveis Quantitativas na Base de Treino',fontsize=25)
cbar = heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=17)
plt.show()

# Analise e tratamento de Outliers 

In [ ]:
#%% analise de outliers das variaveis na base de treino 


#Outliers podem prejudicar MLPs, pois distorcem o treinamento e causam grandes gradientes, 
#o que pode dificultar a minimizacao da funcao de perda e levar a sobreajuste (overfitting). Isso afeta a capacidade de generalizacao do modelo. 
#é importante remover ou transformar outliers e usar tecnicas de regularizacao para mitigar isso 

#lembrando que isso so ocorre com variaveis numericas, vai ficar claro no barplot, onde poderemos ver as medidas de posicao 
variaveis = [
    'CreditScore',
    'Age',
    'Tenure',
    'Balance',
    'NumOfProducts',
    'EstimatedSalary',
    'Satisfaction Score',
    'Point Earned' 
]

# definindo tamenho dos subplots 
plt.figure(figsize=(12, 8))

# boxplots para cada variavel
for i, var in enumerate(variaveis):
    plt.subplot(3, 3, i + 1)  # 3 linhas e 3 colunas
    sns.boxplot(y=teste_multco_treino[var],
               boxprops=dict(facecolor='lightblue'))  # Cor interna do boxplot)  
    plt.title(f'Boxplot {var}', fontsize=12)
    
#  título
plt.suptitle('Análise de Outliers nas Variáveis(treino) - antes de "winsorization" ', fontsize=20)

# Ajuste de layout
plt.tight_layout(rect=[0, 0, 1, 0.95]) 
plt.show()



##aplicando procedimento de truncamento ou winsorization nas variaveis com outliers

#O procedimento de truncamento ou winsorization consiste em limitar os valores extremos de um conjunto de dados 
#(vai ficar claro nas duas aplicacoes de barplot a seguir ). 
#Para isso, e calculado um intervalo de valores aceitos com base no primeiro quartil (Q1) e no terceiro quartil (Q3)
#Valores abaixo do limite inferior ou acima do limite superior sao ajustados para os respectivos limites, corrigindo assim os outliers 
#"sem perda" dos dados originais


# Função que aplica winsorization
def tratar_outliers(df, coluna):
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    # Substitui outliers pelo limite inferior ou superior
    df[coluna] = np.where(df[coluna] < limite_inferior, limite_inferior, df[coluna])
    df[coluna] = np.where(df[coluna] > limite_superior, limite_superior, df[coluna])



# Aplicando a função nas variáveis 
variaveis_para_tratar = ['Age', 'CreditScore', 'NumOfProducts']
for variavel in variaveis_para_tratar:
    tratar_outliers(teste_multco_treino, variavel)


# subplot
plt.figure(figsize=(12, 8))



# boxplots separados para cada variável
for i, var in enumerate(variaveis):
    plt.subplot(3, 3, i + 1)  # 3 linhas e 3 colunas
    sns.boxplot(y=teste_multco_treino[var],
               boxprops=dict(facecolor='green'))  # Cor interna do boxplot)  # Usar o nome da variável diretamente
    plt.title(f'Boxplot {var}', fontsize=12)


# título geral
plt.suptitle('Análise de Outliers nas Variáveis(treino) - depois de "winsorization" ', fontsize=20)


# Ajustando layout
plt.tight_layout(rect=[0, 0, 1, 0.95])  
plt.show()

In [ ]:
# Separando novamente as bases de treino e teste depois de tratar os outliers APENAS na base de TREINO  

#treino
X_train=teste_multco_treino.drop('Exited', axis=1)
y_train=teste_multco_treino['Exited']

#teste
base_corrige_teste = pd.concat([X_test,y_test], axis = 1)
X_test=base_corrige_teste.drop('Exited', axis=1)
y_test=base_corrige_teste['Exited']


print('---------------------------------------')
print(X_train.isnull().sum())
print('---------------------------------------')
print('---------------------------------------')
print(X_train.dtypes)
print('---------------------------------------')
print('---------------------------------------')
print(y_train.value_counts())
print('---------------------------------------')

# Modelagem: aplicando MLP com Pytorch

O PyTorch é uma biblioteca de aprendizado profundo de código aberto, usada principalmente para o treinamento de redes neurais artificiais. Ele oferece flexibilidade e eficiência, permitindo a criação de modelos complexos e o treinamento acelerado com GPUs. O PyTorch é popular entre os pesquisadores devido à sua abordagem dinâmica de construção de grafos computacionais, facilitando a experimentação.

Para mais informações, você pode acessar a documentação oficial do PyTorch:
https://pytorch.org/docs/stable/


OBS: o uso de GPU precisa ser corrigido, a conversao dos tensores esta apresentando problemas. 
Visto isso, o codigo tambem executa em CPU, basta nao ativar a GPU (CUDA) que ele seleciona a CPU automaticamente.



In [ ]:
print('------------------------')
print("MLP - Multi-Layer Perceptron ")
print('------------------------')

# Desativando os warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module='torch')
warnings.filterwarnings("ignore", category=UserWarning, module='optuna')
warnings.filterwarnings("ignore", category=FutureWarning, module='optuna')
warnings.filterwarnings('ignore', category=DeprecationWarning)
logging.getLogger("optuna").setLevel(logging.CRITICAL)


# início
start_time_utc = datetime.utcnow() - timedelta(hours=3)
print('------------------------')
print("Início:", start_time_utc)
print('------------------------')


######################################## PRE-PROCESSAMENTO E PREPARACAO NOS DADOS ################################################################### 


# Definir as variáveis de treinamento
X_train = teste_multco_treino.drop('Exited', axis=1)
y_train = teste_multco_treino['Exited']

# Verificar e corrigir desalinhamento de índices em X_test e y_test antes da concatenação
# Verificar e corrigir desalinhamento de índices em X_test e y_test antes da concatenação
if not X_test.index.equals(y_test.index):
    print("Índices de X_test e y_test não estavam alinhados. Realinhando y_test.")
    y_test = y_test.loc[X_test.index]
else:
    print("Índices de X_test e y_test já estavam alinhados.")

# Concatenar os dados corrigidos para criar a base de teste
base_corrige_teste = pd.concat([X_test, y_test], axis=1)

# Redefinir X_test e y_test com base na base corrigida
X_test = base_corrige_teste.drop('Exited', axis=1)
y_test = base_corrige_teste['Exited']


# Configurando dispositivo para PyTorch, aqui ele verifica a disponibilidade de GPU do kaggle e usa se tiver 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Testando opções de scaler (essas outras foram testadas mas nao foram melhores a principio)
scaler = StandardScaler()
#scaler = MinMaxScaler()
#scaler = RobustScaler()

# Escalonando os dados de treinamento e o conjunto de teste 
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertendo para tensores PyTorch e movendo para o dispositivo
bar = tqdm(total=1, desc="Processando tensores", ncols=100, position=0) #barra de progresso plotada, ajuda a monitorar

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)  
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)  

bar.update(1) #incrementa barra
bar.close()#fecha barra

# Verificando os shapes dos dados depois de tensores criados e dispositivo ativado
print(f"X_train_tensor shape: {X_train_tensor.shape} | device: {X_train_tensor.device}")
print(f"y_train_tensor shape: {y_train_tensor.shape} | device: {y_train_tensor.device}")
print(f"X_test_tensor shape: {X_test_tensor.shape} | device: {X_test_tensor.device}")
print(f"y_test_tensor shape: {y_test_tensor.shape} | device: {y_test_tensor.device}")




################################################# MODELAGEM ################################################################### 

bar = tqdm(total=1, desc="Criando modelo", ncols=100, position=0)# Inicializando a barra de progresso

# Função para salvar o modelo usando o método 'torch.save'
def save_best_model(model, filepath):
    torch.save(model.state_dict(), filepath)
    print(f"Modelo salvo em {filepath}")


# Definir a classe MLP do modelo
class MLP(nn.Module):
    def __init__(self, hidden_layer_sizes, activation, dropout_rate=0.2):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        input_size = X_train_tensor.shape[1]  
        for units in hidden_layer_sizes:
            self.layers.append(nn.Linear(input_size, units))
            self.layers.append(nn.Dropout(p=dropout_rate))  #Aplica dropout
            input_size = units
        self.output = nn.Linear(input_size, 1)  #Camada de saída
        self.activation_fn = self.get_activation_function(activation)
    
    def forward(self, x):
        for layer in self.layers:
            x = self.activation_fn(layer(x))  # Ativação após cada camada linear
        x = self.output(x)  
        return x

    def get_activation_function(self, activation):
        activation_dict = {
            'relu': torch.relu,
            'tanh': torch.tanh,
            'sigmoid': torch.sigmoid,
            'selu': torch.selu,
            'gelu': torch.nn.functional.gelu,
            'leaky_relu': torch.nn.functional.leaky_relu,
            'swish': torch.nn.functional.silu,
        }
        return activation_dict.get(activation, torch.relu)  # Default e Relu so pra morrer de certeza que nao va da erro e a relu e a mais comum

# Função para criar o modelo
def create_model(
    hidden_layer_sizes,
    activation,
    solver,
    alpha,
    learning_rate_init,
    max_iter,
    batch_size,
    momentum,
    learning_rate_scheduler,
    weight_decay,
    step_size,
    gamma,
    early_stopping,
    validation_fraction,
    dropout_rate,
    patience,
    imbalance_ratio=1.0  
):

    # Calcular pesos da classe para aplicacao da funcao de peso para classe minoritaria (BCEWithLogitsLoss)
    count_neg = (y_train_tensor == 0).sum().item()  # Número da classe 0
    count_pos = (y_train_tensor == 1).sum().item()  # Número da classe 1
    pos_weight = torch.tensor([count_neg / count_pos], device=device)  # Ponderação para a classe minoritária
        
    # Instanciar o modelo
    model = MLP(hidden_layer_sizes, activation, dropout_rate).to(device)
    
    # Função de perda com peso para a classe minoritária
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    # Configurar o otimizador
    optimizers = {
        'adam': optim.Adam(model.parameters(), lr=learning_rate_init, weight_decay=weight_decay),
        'sgd': optim.SGD(model.parameters(), lr=learning_rate_init, momentum=momentum, weight_decay=weight_decay),
        'rmsprop': optim.RMSprop(model.parameters(), lr=learning_rate_init, weight_decay=weight_decay),
        'nadam': optim.NAdam(model.parameters(), lr=learning_rate_init, weight_decay=weight_decay),
    }
    optimizer = optimizers.get(solver, optim.Adam(model.parameters(), lr=learning_rate_init, weight_decay=weight_decay))
    
    # Configurar o scheduler
    schedulers = {
        'StepLR': lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma),
        'CosineAnnealingLR': lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_iter),
        'ReduceLROnPlateau': lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=gamma, patience=16, verbose=True),
        'CosineAnnealingWarmRestarts': lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    }
    scheduler = schedulers.get(learning_rate_scheduler, None)

    return model, criterion, optimizer, scheduler

bar.update(1)  
bar.close()


# Função de objetivo para o Optuna
bar = tqdm(total=1, desc="Criando hiperparametros Optuna E validação cruzada", ncols=100, position=0) 

bar2=None
def objective(trial):
    global bar2
    try:
        # Definindo os parâmetros a serem otimizados
        hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [(100, 50), (200, 100), (300, 200), (500, 400), (600, 500), (700, 600), (800, 700)])
        activation = trial.suggest_categorical('activation', ['leaky_relu', 'swish','gelu'])
        solver = trial.suggest_categorical('solver', ['adam', 'sgd','rmsprop'])
        alpha = trial.suggest_loguniform('alpha', 1e-4, 1e-2)
        learning_rate_init = trial.suggest_loguniform('learning_rate_init', 1e-6, 1e-4)
        max_iter = trial.suggest_categorical('max_iter', [2000, 3000, 4000, 5000, 6000])
        batch_size = trial.suggest_categorical('batch_size', [256, 512])
        momentum = trial.suggest_uniform('momentum', 0.7, 0.9)
        learning_rate_scheduler = trial.suggest_categorical('learning_rate_scheduler', ['ReduceLROnPlateau','StepLR', None])
        weight_decay = trial.suggest_loguniform('weight_decay', 1e-4, 1e-2)
        step_size = trial.suggest_int('step_size', 20, 60)
        gamma = trial.suggest_uniform('gamma', 0.5, 0.999)
        early_stopping = trial.suggest_categorical('early_stopping', [True])
        validation_fraction = trial.suggest_uniform('validation_fraction', 0.2, 0.5)
        dropout_rate = trial.suggest_uniform('dropout_rate', 0.5, 0.8)
        patience = trial.suggest_int('patience', 10, 20)


        
        # Criação do modelo
        model, criterion, optimizer, scheduler = create_model(
            hidden_layer_sizes=hidden_layer_sizes,
            activation=activation,
            solver=solver,
            alpha=alpha,
            learning_rate_init=learning_rate_init,
            max_iter=max_iter,
            batch_size=batch_size,
            momentum=momentum,
            learning_rate_scheduler=learning_rate_scheduler,
            weight_decay=weight_decay,
            step_size=step_size,
            gamma=gamma,
            early_stopping=early_stopping,
            validation_fraction=validation_fraction,
            dropout_rate=dropout_rate,
            patience=patience)
        
        
        # Função de treinamento e validação cruzada

        def train_and_evaluate_cross_validation(epochs, patience):
            skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            auc_scores = []
            recall_scores = []
            f1_scores = []
            
            # Instanciando o undersampler
            #undersampler = RandomUnderSampler(random_state=42)
            
            # Instanciando o SMOTE
            # smote = SMOTE(random_state=42)  
  
            # Instanciando o SMOTE ADASYN
            adasyn = ADASYN(random_state=42)
        
            for train_index, val_index in skf.split(X_train_tensor.cpu().numpy(), y_train_tensor.cpu().numpy()):
                X_train_cv, X_val_cv = X_train_tensor[train_index].to(device), X_train_tensor[val_index].to(device)
                y_train_cv, y_val_cv = y_train_tensor[train_index].to(device), y_train_tensor[val_index].to(device)

                #  # aplicar undersampler :
                #X_train_cv_res, y_train_cv_res = undersampler.fit_resample(X_train_cv.cpu().numpy(), y_train_cv.cpu().numpy())
                
                # aplicar SMOTE :
                # X_train_cv_res, y_train_cv_res = smote.fit_resample(X_train_cv.cpu().numpy(), y_train_cv.cpu().numpy())
                
                # aplicar SMOTE ADASYN:
                X_train_cv_res, y_train_cv_res = adasyn.fit_resample(X_train_cv.cpu().numpy(), y_train_cv.cpu().numpy())

                
                X_train_cv_res = torch.tensor(X_train_cv_res).float().to(device)
                y_train_cv_res = torch.tensor(y_train_cv_res).float().to(device)
        
                # Treinamento do modelo
                model.to(device)
                model.train()
                train_dataset = TensorDataset(X_train_cv_res, y_train_cv_res)
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
                # Implementação de Early Stopping
                best_val_loss = float('inf')
                patience_counter = 0
                for epoch in range(epochs):
                    model.train()
                    for X_batch, y_batch in train_loader:
                        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                        optimizer.zero_grad()
                        output = model(X_batch).squeeze()
                        loss = criterion(output, y_batch.to(device))
                        loss.backward()
                        optimizer.step()
        
                    # Avaliação da perda de validação para early stopping
                    model.eval()
                    with torch.no_grad():
                        val_preds = model(X_val_cv).squeeze().cpu().numpy()
                        val_labels = y_val_cv.cpu().numpy()
                        val_loss = criterion(torch.tensor(val_preds), torch.tensor(val_labels).float()).item()
                        if val_loss < best_val_loss:
                            best_val_loss = val_loss
                            patience_counter = 0
                        else:
                            patience_counter += 1
                        if patience_counter >= patience:
                            break
                
                    # Avaliação no conjunto de validação após treinamento
                    model.eval()
                    with torch.no_grad():
                        val_logits = model(X_val_cv).squeeze().cpu().numpy()  # Obter logits brutos
                        val_labels = y_val_cv.cpu().numpy()
                        
                        # função sigmoide para converter logits em probabilidades
                        val_preds = torch.sigmoid(torch.tensor(val_logits)).numpy()
                        
                        # Fixando o threshold em 0.5
                        threshold = 0.5
                        
                        # Convertendo probabilidades em previsões binárias
                        val_preds_binary = (val_preds > threshold).astype(int)
                        
                        #  métricas com as previsões ajustadas
                        auc = roc_auc_score(val_labels, val_preds)  
                        recall = recall_score(val_labels, val_preds_binary)
                        f1 = f1_score(val_labels, val_preds_binary)
                        
                        # Armazenando as métricas
                        f1_scores.append(f1)
                        auc_scores.append(auc)
                        recall_scores.append(recall)
        
            return np.mean(auc_scores), np.mean(recall_scores), np.mean(f1_scores)
        
        #validação cruzada
        mean_auc, mean_recall, mean_f1 = train_and_evaluate_cross_validation(epochs=250, patience=16)

        
        #métrica composta
        score = 0.85 * mean_auc + 0.15 * mean_recall
        
        # resultados ao Optuna
        trial.report(score, 0)  #  step 0
        trial.report(mean_recall, 1)  #step 1
        trial.report(mean_f1, 2)     #  step 2
        
        if trial.should_prune():
            print(f"Trial {trial.number} foi interrompido devido ao pruning.")
            raise optuna.exceptions.TrialPruned()
        
        # Salvando o modelo no trial para ser acessado após a otimização
        trial.set_user_attr('model', model)
        
        
        bar2.update(1)
        
        # Retorna a métrica composta
        return score 

    


        
    except Exception as e:
        # Captura o tipo de erro e a mensagem de erro
        error_type = type(e).__name__
        error_message = str(e)
        
        # Captura o traceback completo para mais detalhes
        error_traceback = traceback.format_exc()
    
        # Exibe o erro completo no console
        print(f"Trial {trial.number} falhou com erro: {error_type}")
        print(f"Mensagem do erro: {error_message}")
        print("Traceback completo:")
        print(error_traceback)
    
        bar2.update(1) # Atualiza a barra de progresso após cada trial
    
        # Retorna valor negativo infinito para continuar a otimização
        return float('-inf')
     
    # Salvar o modelo após a otimização
    # torch.save(model.state_dict(), 'best_model.pth')  ---- salva apenas os parametros, 
    #se o modelo for ser usado fora desse processamento/codigo
    # o ideal é salva-lo em de outra forma (torch.save(model, 'best_model.pth'))
    torch.save(model.state_dict(), 'best_model.pth')

    
bar.update(1)
bar.close()  # Fechar a barra




########################## Definindo o estudo do Optuna############################################
n_trials = 35
bar2 = tqdm(total=n_trials, desc="Otimização em andamento", ncols=100, position=0)

study = optuna.create_study(direction="maximize") 
study.optimize(objective, n_trials=n_trials, catch=(Exception,))

best_model = study.best_trial.user_attrs['model']

# Salvando o melhor modelo otimizado
save_best_model(best_model, 'best_model.pth')

# Resultados
best_params = study.best_params
print(f"Melhores parâmetros: {best_params}")

bar2.close()
bar.close()

print('######## fim de estudo do OPTUNA ###########')




# Inicializando a barra de progresso
epochs = 250 #quantidade de epocas



bar = tqdm(total=epochs, desc="########### Treinamento final ##############", ncols=100, position=0)


# Pré-processamento: Divisão dos dados e normalização
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalização dos dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Convertendo os dados para tensores do PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)

# Criando os datasets para treinamento e validação
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Criando os DataLoaders
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=best_params['batch_size'], shuffle=False, num_workers=4)

# Função de treinamento de uma época
def train_one_epoch(model, train_loader, optimizer, criterion, device, scheduler=None):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()

        # Clipping de gradientes para evitar explosão
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        running_loss += loss.item() * X_batch.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    
    if scheduler and not isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
        scheduler.step()
    
    return epoch_loss

# Função de validação
def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            val_loss += loss.item() * X_batch.size(0)
    return val_loss / len(val_loader.dataset)

# Inicializando o modelo, otimizador e scheduler
model, criterion, optimizer, scheduler = create_model(**best_params)
model.to(device)  # Movendo o modelo para o dispositivo

# Listas para armazenar as perdas
train_losses = []
val_losses = []
best_loss = float('inf')

# Loop de treinamento final
for epoch in range(epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device, scheduler)
    val_loss = validate(model, val_loader, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    # Ajuste do scheduler com a perda de validação, se aplicável
    if scheduler and isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
        scheduler.step(val_loss)

    # Salvando o melhor modelo
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth") #aqui salva o OrderedDict (sopmente os parametros, bom pra usar no mesmo codigo enquanto reina e testa o 80/20)
        torch.save(model, 'best_model_inteiro.pth') #salva o modelo INTEIRO com dict e tudo, bom para aplicar o modelo em producao
        torch.save(scaler, 'scaler.pth')  # Salvar scaler usado no treino


    bar.update(1)

bar.close()

# Carregar o melhor modelo após o treinamento
model.load_state_dict(torch.load("best_model.pth"))

# Plotando as perdas por época
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss', color='blue', linestyle='-', marker='o')
plt.plot(val_losses, label='Validation Loss', color='orange', linestyle='-', marker='o')
plt.title('Training and Validation Loss per Epoch', fontsize=16)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

print("###########################Treinamento final concluído com sucesso!######################################")
print(f"Menor perda de validação: {best_loss:.4f}")



# Avaliação final
print("###########################Avaliando no conjunto de teste!######################################")
# Carregando o melhor modelo
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Avaliando no conjunto de teste
with torch.no_grad():
    y_pred_logits = model(X_test_tensor.to(device)).cpu().numpy().squeeze()  # Isso são logits
    y_pred_prob_test = torch.sigmoid(torch.tensor(y_pred_logits)).cpu().numpy()  # Aplicando a sigmoide para obter probabilidades



#threshold 
# Fixando o threshold em 0.5
threshold = 0.5
y_pred_class_test = (y_pred_prob_test > threshold).astype(int)

# Calculando métricas
roc_auc = roc_auc_score(y_test, y_pred_prob_test)
accuracy = accuracy_score(y_test, y_pred_class_test)
precision = precision_score(y_test, y_pred_class_test)
recall = recall_score(y_test, y_pred_class_test)
f1 = f1_score(y_test, y_pred_class_test)

# Exibindo métricas
print(f"ROC AUC (Teste): {roc_auc:.4f}")
print(f"Acurácia (Teste): {accuracy:.4f}")
print(f"Precisão (Teste): {precision:.4f}")
print(f"Recall (Teste): {recall:.4f}")
print(f"F1-Score (Teste): {f1:.4f}")


conf_matrix = confusion_matrix(y_test, y_pred_class_test)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Não Exited', 'Exited'], yticklabels=['Não Exited', 'Exited'])
plt.xlabel('Predição')
plt.ylabel('Real')
plt.title('Matriz de Confusão no teste')
plt.show()



# Cálculo da curva ROC para o conjunto de teste
fpr_test, tpr_test, _ = roc_curve(y_test, y_pred_prob_test)
roc_auc_test = auc(fpr_test, tpr_test)

# Gráfico ROC teste
plt.figure(figsize=(8, 6))
plt.plot(fpr_test, tpr_test, color='red', label=f'ROC Curve - Teste (AUC = {roc_auc_test:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.title('Curva ROC - Teste')
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.legend(loc='lower right')
plt.show()

In [ ]:
################FAZENDO PREDIÇÕES SIMULANDO UMA APLICAÇÃO EM PRODUÇÃO EM DADOS NOVOS ##############################

#NO CASO COMO NAO TEM DADOS NOVOS, FOI APLICADO NA BASE INTEIRA ORIGINAL
print("##########Simulando predições em 'produção'#####################")

# Carregar o modelo treinado inteiro e em modo de avaliação
model = torch.load('best_model_inteiro.pth')
model.eval()  

# Carregar o scaler utilizado no treinamento
scaler = torch.load('scaler.pth')

# Carregar os novos dados
df_new = base_corrige_teste

#df_new = df_new.head(5000) #para testar em apenas 5000 observacoes 

# Separar as features x e ^y
X_new = df_new.drop(columns=['Exited'])  
y_true = df_new['Exited'].values  # Valores reais para avaliação

# Aplicar o mesmo pré-processamento (normalização)
X_scaled = scaler.transform(X_new)

# Converter para tensor PyTorch
input_tensor = torch.tensor(X_scaled, dtype=torch.float32)

# Obter predições do modelo
with torch.no_grad():
    y_pred_logits = model(input_tensor).numpy()  # Logits previstos
    y_pred_prob = torch.sigmoid(torch.tensor(y_pred_logits)).numpy()  # Aplicando sigmoide para obter probabilidades

# Converter probabilidades em classes (0 ou 1)
y_pred_class = (y_pred_prob > 0.5).astype(int)

# Calcular matriz de confusão
conf_matrix = confusion_matrix(y_true, y_pred_class)

# Visualizar matriz de confusão 
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Greens', xticklabels=[0,1], yticklabels=[0,1])
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Dados de simulacao de Producao')
plt.show()

# Calcular a curva ROC e a AUC
fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plotar a curva ROC
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')  # Linha de referência
plt.xlabel('Falso Positivo Rate')
plt.ylabel('Verdadeiro Positivo Rate')
plt.title('Curva ROC - Dados de simulacao de Producao')
plt.legend()
plt.show()

# Calcular a acurácia total
accuracy_positive = accuracy_score(y_true, y_pred_class)
print(f"Acurácia total - Dados de simulacao de Producao: {accuracy_positive * 100:.2f}%")

# Calcular o recall da classe positiva
positive_class_recall = recall_score(y_true, y_pred_class)
print(f"Recall da classe positiva - Dados de simulacao de Producao: {positive_class_recall * 100:.2f}%")


# Fim
end_time_utc = datetime.utcnow() - timedelta(hours=3)
print('------------------------')
print("Início:", start_time_utc)
print("Fim:   ", end_time_utc)
print('------------------------')